In [24]:
# Import libraries for data wrangling, preprocessing and visualization
import numpy 
import pandas as pd
import matplotlib.pyplot as pltS
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [25]:
# Importing libraries for building the neural network
from keras.models import SequentialS
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [50]:
# Read data file
data = pd.read_csv("data.csv")

In [51]:
data = data.drop(columns = ["Unnamed: 0", 'Fee', 'State', 'RescuerID',
                           'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'BreedName_1', 'BreedName_2',
                           'Description_Character_Count', 'Description_Word_Count', "Name", "Age"])

In [52]:
data.columns

Index(['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3',
       'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized',
       'Health', 'Quantity', 'AdoptionSpeed', 'Unnamed', 'MixedBreed',
       'Word_25', 'Word_5', 'Word_75', 'Word_100', 'Puppy', 'Adult', 'Senior',
       'No_Fee', 'Fee_25', 'Fee_5', 'Fee_75', 'Fee_100', 'Photo_Score',
       'Sentiment_Score', 'Sentiment_Magnitude'],
      dtype='object')

In [53]:
data = pd.get_dummies(data, columns = ["Type", "Gender", 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
                                        'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
                                        'Sterilized', 'Health'])

In [54]:
data.columns

Index(['Quantity', 'AdoptionSpeed', 'Unnamed', 'MixedBreed', 'Word_25',
       'Word_5', 'Word_75', 'Word_100', 'Puppy', 'Adult',
       ...
       'Vaccinated_3', 'Dewormed_1', 'Dewormed_2', 'Dewormed_3',
       'Sterilized_1', 'Sterilized_2', 'Sterilized_3', 'Health_1', 'Health_2',
       'Health_3'],
      dtype='object', length=367)

In [56]:
# Select the columns to use for prediction in the neural network
Y = data["AdoptionSpeed"]

X = data.drop(columns = "AdoptionSpeed")

In [57]:
# Diagnosis values are strings. Changing them into numerical values using LabelEncoder.
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [65]:
# Baseline model for the neural network. We choose a hidden layer of 10 neurons. The lesser number of neurons helps to eliminate the redundancies in the data and select the more important features.
def create_baseline():
    model = Sequential()
    model.add(Dense(10, input_dim=366, kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

In [66]:
# Evaluate model using standardized dataset. 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 35.88% (1.39%)
